This project is a text classification task, conducted on the Kaggle platform using the NLP Disaster Tweets dataset. The goal is to build a machine learning model that can accurately classify whether a tweet is related to a real disaster or not. 

The dataset consists of a training set with approximately 7,600 labeled tweets and a test set with approximately 3,200 unlabeled tweets. The tweets are preprocessed and combined into a single string, which is then tokenized and padded to a fixed length. 

The model used for classification is a recurrent neural network (RNN) with a long short-term memory (LSTM) layer. The input to the model is a sequence of tokenized and padded tweets, which are fed through the embedding layer and into the LSTM layer. The LSTM layer is followed by a dense layer with a ReLU activation function, dropout regularization, and a final output layer with a sigmoid activation function to produce binary classification predictions. 

The model is trained using the binary cross-entropy loss function and the Adam optimizer. The accuracy of the model is evaluated on a validation set, and the final predictions are made on the test set. The accuracy of the predictions is calculated using the F1 score.

Kaggle dataset Ref:https://www.kaggle.com/competitions/nlp-getting-started/data?select=train.csv

EDA 

In [10]:
import pandas as pd
# Load data
train_data = pd.read_csv("N:/train.csv")

# Replace missing values in keyword and location columns with "unknown"
train_data['keyword'] = train_data['keyword'].fillna('unknown')
train_data['location'] = train_data['location'].fillna('unknown')
print(train_data.head())


   id  keyword location                                               text   
0   1  unknown  unknown  Our Deeds are the Reason of this #earthquake M...  \
1   4  unknown  unknown             Forest fire near La Ronge Sask. Canada   
2   5  unknown  unknown  All residents asked to 'shelter in place' are ...   
3   6  unknown  unknown  13,000 people receive #wildfires evacuation or...   
4   7  unknown  unknown  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


Now we'll create an LSTM network using Tensorflow:

In [11]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
sequences = tokenizer.texts_to_sequences(train_data['text'])

# Pad sequences
max_len = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# Prepare target data
target = train_data['target'].values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, target, test_size=0.2, random_state=42)

# LSTM model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(lr=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

# Make predictions on the validation set
y_pred = (model.predict(X_val) > 0.5).astype("int32")

# Calculate the accuracy
accuracy = np.sum(y_pred.reshape(-1) == y_val) / len(y_val)
print(f"Accuracy: {accuracy * 100:.2f}%")


Epoch 1/10


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


96/96 [==============================] - 5s 41ms/step - loss: 0.6286 - accuracy: 0.6460 - val_loss: 0.4781 - val_accuracy: 0.8004
Epoch 2/10
96/96 [==============================] - 3s 36ms/step - loss: 0.3734 - accuracy: 0.8571 - val_loss: 0.4698 - val_accuracy: 0.7932
Epoch 3/10
96/96 [==============================] - 4s 37ms/step - loss: 0.2074 - accuracy: 0.9340 - val_loss: 0.5934 - val_accuracy: 0.7768
Epoch 4/10
96/96 [==============================] - 3s 36ms/step - loss: 0.1127 - accuracy: 0.9683 - val_loss: 0.7746 - val_accuracy: 0.7557
Epoch 5/10
96/96 [==============================] - 4s 37ms/step - loss: 0.0649 - accuracy: 0.9837 - val_loss: 0.7167 - val_accuracy: 0.7820
Epoch 6/10
96/96 [==============================] - 4s 37ms/step - loss: 0.0521 - accuracy: 0.9883 - val_loss: 0.9577 - val_accuracy: 0.7577
Epoch 7/10
96/96 [==============================] - 3s 36ms/step - loss: 0.0307 - accuracy: 0.9929 - val_loss: 1.2904 - val_accuracy: 0.7557
Epoch 8/10
96/96 [======

I attempted to increase the LSTM parameters and retrain the model. Specifically, I increased the LSTM layer size to 128 and the size of the dense layer to 64. Dropout regularization was used in both the LSTM and dense layers. 

The model was compiled with the Adam optimizer and binary cross-entropy loss function. It was then trained on the training set with 10 epochs and a batch size of 64. The model was evaluated on a validation set, and the accuracy of the predictions was calculated using the F1 score. 

After training the model, the accuracy of the predictions on the validation set was evaluated. The results showed that the accuracy of the model was improved compared to the previous model.

In [12]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(lr=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

# Make predictions on the validation set
y_pred = (model.predict(X_val) > 0.5).astype("int32")

# Calculate the accuracy
accuracy = np.sum(y_pred.reshape(-1) == y_val) / len(y_val)
print(f"Accuracy: {accuracy * 100:.2f}%")

Epoch 1/10
96/96 [==============================] - 8s 63ms/step - loss: 0.5834 - accuracy: 0.6936 - val_loss: 0.4697 - val_accuracy: 0.7859
Epoch 2/10
96/96 [==============================] - 6s 61ms/step - loss: 0.3277 - accuracy: 0.8737 - val_loss: 0.4713 - val_accuracy: 0.8011
Epoch 3/10
96/96 [==============================] - 6s 60ms/step - loss: 0.1699 - accuracy: 0.9465 - val_loss: 0.6300 - val_accuracy: 0.7689
Epoch 4/10
96/96 [==============================] - 6s 62ms/step - loss: 0.0889 - accuracy: 0.9713 - val_loss: 0.7744 - val_accuracy: 0.7584
Epoch 5/10
96/96 [==============================] - 6s 63ms/step - loss: 0.0572 - accuracy: 0.9836 - val_loss: 0.7555 - val_accuracy: 0.7466
Epoch 6/10
96/96 [==============================] - 6s 63ms/step - loss: 0.0492 - accuracy: 0.9880 - val_loss: 0.7724 - val_accuracy: 0.7695
Epoch 7/10
96/96 [==============================] - 6s 62ms/step - loss: 0.0312 - accuracy: 0.9913 - val_loss: 1.0332 - val_accuracy: 0.7341
Epoch 8/10
96

In this project, we attempted to improve the accuracy of a machine learning model for text classification using the NLP Disaster Tweets dataset. We first trained a baseline model with a standard LSTM architecture and achieved an accuracy of 76.30%. 

We then attempted to improve the performance of the model by increasing the size of the LSTM layer and dense layer. However, the results showed that simply increasing the size of the LSTM layer did not significantly improve the accuracy of the model.

To improve the accuracy of the model, we need to consider the overall problem and explore other methods beyond the basic LSTM architecture. One approach could be to incorporate additional features, such as location information, into the model to enhance its performance. Another option could be to explore more advanced RNN architectures, such as a bidirectional LSTM or a gated recurrent unit (GRU), which may be more effective for this specific task.

In summary, improving the accuracy of a text classification model requires a holistic approach that considers multiple factors, including feature engineering, model architecture, and hyperparameter tuning. By carefully exploring and experimenting with these different factors, we can build more effective and accurate models for text classification tasks.

For Kaggle below

In [16]:
# Load the test dataset
test_df = pd.read_csv('N://test.csv')

# Get the IDs and texts from the test dataset
ids = test_df['id'].values
texts = test_df['text'].values

# Tokenize the text
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')


# Make predictions on the test dataset
y_pred = model.predict(padded_sequences).flatten()
y_pred = [0 if pred < 0.5 else 1 for pred in y_pred]
# Create a DataFrame with the predicted values and IDs
result_df = pd.DataFrame({'id': ids, 'target': y_pred})

# Save the DataFrame to a CSV file
result_df.to_csv('N://output.csv', index=False)

102/102 [==============================] - 1s 6ms/step
